In [4]:
# import the necessary packages
from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils

In [2]:
import cv2

In [5]:
import pandas as pd

In [4]:
import pytesseract

In [5]:
from pytesseract import Output

In [6]:
import math

In [7]:
# import the necessary packages
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest differenceimport cv2

import numpy as np

import pandas as pd

import pytesseract

from pytesseract import Output

import math
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect

IndentationError: unexpected indent (3142671829.py, line 26)

In [8]:
def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped

In [7]:
# load the image and compute the ratio of the old height
# to the new height, clone it, and resize it
image = cv2.imread("IMG_3222.jpg")
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)
# convert the image to grayscale, blur it, and find edges
# in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)
# show the original image and the edge detected image
print("STEP 1: Edge Detection")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

STEP 1: Edge Detection


In [79]:
# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_TC89_KCOS)
cnts = imutils.grab_contours(cnts)

In [80]:
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

In [81]:
# loop over the contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.05 * peri, True)
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break

In [82]:
# show the contour (outline) of the piece of paper
print("STEP 2: Find contours of paper")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

STEP 2: Find contours of paper


In [54]:
# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)
# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255
# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv2.imshow("Original", imutils.resize(orig, height = 650))
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
cv2.waitKey(0)

STEP 3: Apply perspective transform


32

In [61]:
#receipt_str = pytesseract.image_to_string(receipt_img,output_type=Output.DICT)
receipt_df = pytesseract.image_to_data(warped,output_type=Output.DATAFRAME,config="--psm 6")

In [66]:
receipt_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,1736,3759,-1,NaN
1,2,1,1,0,0,0,0,19,1728,3575,-1,NaN
2,3,1,1,1,0,0,0,15,1736,3579,-1,NaN
3,4,1,1,1,1,0,46,19,1671,123,-1,NaN
4,5,1,1,1,1,1,34,15,999,94,8,Oe
...,...,...,...,...,...,...,...,...,...,...,...,...
128,5,1,1,1,23,5,910,3521,60,50,92,1
129,4,1,1,1,24,0,80,3530,818,64,-1,NaN
130,5,1,1,1,24,1,80,3532,307,62,13,Ferminal:
131,5,1,1,1,24,2,434,3530,92,54,86,101


In [67]:
receipt_df['text'] = receipt_df['text'].astype('string')

In [68]:
receipt_df['text']

0           <NA>
1           <NA>
2           <NA>
3           <NA>
4             Oe
         ...    
128            1
129         <NA>
130    Ferminal:
131          101
132    Restauran
Name: text, Length: 133, dtype: string

In [69]:
for i in range(len(receipt_df['text'])):
    if np.where(pd.isna(receipt_df['text'][i]),False,True) and np.where(receipt_df['text'][i] != 'nan' or receipt_df['text'][i].isspace(),True,False):
        print(receipt_df['text'][i])# , receipt_dict['conf'][i])

Oe
GUT
PE
Ae
We
Ba
wh
TL
ty
IEE
aE
EC
WIRE
orm
ae
IH]
RAR
©
BOM
Bee
OP
EE
A
Awd
Wage
aps,
Cy
PCIE
NN
Srighion
i960
Lower
Kings
Fload
Brighton,
BNI2LN
VAT:
185
080
903
30/10/2022
14-13-02
Table/TAB
No:
7
7
1
|.
Flat
White
2.95
_
|
Gordons
Gin
25m
3.75
|
Soup
of
the
Day
7.00
|
small
Fish
n
Chips
8.00
|
Scampi
15.00
10%
Service
Charge
|
£3.87
AMOUNTOUE:
£46.89
ee
ei
ee
eee
eet
yee
SI
RR
SOS
SS,
RT
VAT
Analysis
|
Receipt
#
15158
Clerk:
Hannah
ce
;
_-
RQastgurant
1
Ferminal:
101
Restauran


In [71]:
n_boxes = len(receipt_df['text'])
for i in range(n_boxes):
    if np.where(pd.isna(receipt_df['text'][i]),False,True) and np.where(receipt_df['text'][i] != 'nan' or receipt_df['text'][i].isspace(),True,False):
        (x, y, w, h) = (receipt_df['left'][i], receipt_df['top'][i], receipt_df['width'][i], receipt_df['height'][i])
        if float(receipt_df['conf'][i]) > 60:  # Check if confidence score is greater than 60
            colour = (0, 255, 0)
        else:
            colour = (0, 0, 255)

        img = cv2.rectangle(warped, (x, y), (x + w, y + h), colour, 2)

In [73]:
cv2.imshow('img', warped)
cv2.waitKey(0)

32